# 0 - Librerías y variables

In [15]:
# Librerías
# ------------------------------------------------------------------------------
import os
import requests
import json

from dotenv import load_dotenv, dotenv_values
load_dotenv()

True

In [3]:
# Variables
# ------------------------------------------------------------------------------
env_vars = dotenv_values()
for key in env_vars.keys():
    print(key)

OPENAI_API_KEY
PROXYCURL_API_KEY
TAVILY_API_KEY
LANGCHAIN_TRACING_V2
LANGCHAIN_ENDPOINT
LANGCHAIN_API_KEY
LANGCHAIN_PROJECT


# 1 - ChatLLM

En este apartado se llama a la API de OpenAI directamente:

In [92]:
from openai import OpenAI

In [93]:
# Inicializar el cliente
client = OpenAI()

messages = [{"role": "user", "content": "¿Cuánto es 1+1?"}]
    
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
)

response.choices[0].message.content

ChatCompletion(id='chatcmpl-BWkfhRmicb3QmFwbJdVQAdrleJCzv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1+1 es igual a 2.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1747145601, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=17, total_tokens=27, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

Si quiero modificar el código por ejemplo para utilizar un modelo LLaMA 3.1 en Ollama:

In [7]:
# Define el endpoint local de Ollama
OLLAMA_URL = "http://localhost:11434/api/chat"

messages = [{"role": "user", "content": "¿Cuánto es 1+1?"}]

response = requests.post(OLLAMA_URL, json={
    "model": "llama3.1",
    "messages": messages,
    "temperature": 0.0,
    "stream": False
})

data = response.json()
print(data["message"]["content"])

La respuesta a la pregunta es 2.


Cada vez que haya un cambio de LLM, tendría que haber un cambio de código.

# 2 - LangChain: Chat API

En este apartado se va a replicar lo se ha hecho en el punto anterior, pero desde la API de LangChain. Adicionalemnte se va ahondar en los siguientes conceptos de LangChain y sus ventajas:
- Modelos
- Prompts Templates
- Output Parsers

## 2.1. - Modelos

En LangChain, los models (modelos) son los componentes que generan texto o responden a mensajes. Son la parte que realmente interactúa con modelos de lenguaje (LLMs) como los de OpenAI, Anthropic, Cohere, etc. LangChain organiza los modelos según lo que hacen. Los más comunes son:

| Tipo de modelo     | Qué hace                                                                    | Clase típica                                |
| ------------------ | --------------------------------------------------------------------------- | ------------------------------------------- |
| **LLM**            | Genera texto a partir de un *prompt plano*                                  | OpenAI, HuggingFaceHub                      |
| **ChatModel**      | Maneja conversaciones con roles (usuario, asistente, sistema)               | ChatOpenAI, ChatAnthropic                   |
| **EmbeddingModel** | Convierte texto en vectores (útiles para búsquedas o comparación semántica) | OpenAIEmbeddings, HuggingFaceEmbeddings     |


LangChain permite llamar a diferentes modelos, con diferentes APIs, de forma agnóstica. Cambiando ligeramente el código, puedo aprovechar una estructura ya creada para apuntar a otro modelo.

In [2]:
import langchain
print(langchain.__version__)

0.3.25


In [8]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

In [9]:
# Crear los mensajes en el formato de LangChain
messages = [
    HumanMessage(content="¿Cuánto es 1+1?")
]

In [100]:
# Llamar a un modelo A
llm = ChatOllama(
    model="llama3.1",
    temperature=0.0
    )

response = llm.invoke(messages)

print(response.content)

La respuesta a la pregunta de "¿Cuánto es 1+1?" es 2.


In [30]:
# Llamar a un modelo B
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

response = llm.invoke(messages)

print(response.content)

1+1 es igual a 2.


La estructura es exactamente la misma, se podría incluso encapsular el codigo en una función y pasar el modelo como parámetro.

In [31]:
def call_llm_model(
    model_name: str,
    temperature: float,
    message: str
) -> str:

    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature
    )

    response = llm.invoke([HumanMessage(content=message)])

    return response.content


respuesta = call_llm_model(
    model_name="gpt-3.5-turbo",
    temperature=0.2,
    message="¿Quién es Andrew Ng?"
)

print(respuesta)

Andrew Ng es un científico de datos, empresario e investigador en inteligencia artificial. Es conocido por ser uno de los pioneros en el campo del aprendizaje profundo y por su trabajo en el desarrollo de algoritmos de aprendizaje automático. Ha sido profesor en la Universidad de Stanford, cofundador de Google Brain y Baidu AI Group, y fundador de la plataforma de educación en línea Coursera. También es cofundador y presidente de Landing AI, una empresa que ayuda a las empresas a implementar soluciones de inteligencia artificial.


En las últimas versiones de LangChain, se ha creado una abstracción para inicilizar modelos de diferentes proveedores desde la misma función.

In [32]:
from langchain.chat_models import init_chat_model

# model = init_chat_model("gpt-4o-mini", model_provider="openai")
# model = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")
# model = init_chat_model("mistral-large-latest", model_provider="mistralai")

## 2.2. - Prompt Templates

Las Prompt Templates (plantillas de prompt) son una herramienta de LangChain que te ayuda a crear automáticamente los mensajes que le envías al modelo de lenguaje, de una forma organizada, flexible y reutilizable.

Cuando trabajas con modelos como GPT, no sueles enviar directamente el texto del usuario al modelo. Normalmente quieres hacer algo más, como:

- Agregar instrucciones específicas (por ejemplo: "Traduce al francés...")
- Dar un contexto adicional al modelo
- Formatear el texto de forma especial
- Usar el mismo formato muchas veces con diferentes datos

Ahí es donde entran las prompt templates.

Piensa en ellas como plantillas con huecos (como los de un formulario). Tú defines una estructura fija y dejas espacios para completar con datos reales más tarde.

In [33]:
template = "Traduce el siguiente texto al {language}: {text}"

LangChain te permite definir esta plantilla y luego rellenarla automáticamente con los valores que quieras:
- `language = "francés"`
- `text = "Hola, ¿cómo estás?"`

los principales templates y su propósito son:

| Prompt Template                    | ¿Para qué sirve?                                                                                                                                   |
| ---------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- |
| `ChatPromptTemplate`               | Plantilla para **modelos de chat**. Permite combinar mensajes (`System`, `Human`, etc.) fácilmente. Ideal para `ChatOpenAI`, `ChatAnthropic`, etc. |
| `PromptTemplate`                   | Plantilla simple de texto plano. Usado con modelos de lenguaje **no conversacionales** como `OpenAI(model="text-davinci-003")`.                    |
| `SystemMessagePromptTemplate`      | Subplantilla usada dentro de `ChatPromptTemplate` para definir el **mensaje del sistema**.                                                         |
| `HumanMessagePromptTemplate`       | Subplantilla usada dentro de `ChatPromptTemplate` para el **mensaje del usuario**                                                                  |
| `MessagesPlaceholder`              | Placeholder especial dentro de `ChatPromptTemplate` para insertar una lista dinámica de mensajes (ej: historial de conversación).                  |
| `AIMessagePromptTemplate`          | Subplantilla para simular un mensaje **anterior del asistente** en el historial.                                                                   |
| `FewShotPromptTemplate`            | Plantilla para tareas de **few-shot learning**. Permite definir ejemplos que se combinan con el input.                                             |
| `ChatMessagePromptTemplate`        | Versión más flexible que permite definir mensajes de un rol personalizado (ej: `"role": "function"`).                                              |
| `FewShotChatMessagePromptTemplate` | Lo mismo que `FewShotPromptTemplate`, pero adaptado para `ChatPromptTemplate`. Útil si quieres ejemplos en formato chat.                           |

In [34]:
from langchain_core.prompts import ChatPromptTemplate


| Método            | ¿Qué construye?                             | ¿Cuándo usarlo?                                 |
| ----------------- | ------------------------------------------- | ----------------------------------------------- |
| `from_template()` | Una sola plantilla de mensaje               | Para casos simples o rápidos                    |
| `from_messages()` | Una conversación entera con múltiples roles | Cuando necesitas dar contexto o varios mensajes |


In [35]:
# Plantilla de traducción
template = "Traduce el siguiente texto al {language}: {text}"

In [36]:
# Para casos sencillos
template_string = "Traduce el siguiente texto al {language}: {text}"
prompt_template = ChatPromptTemplate.from_template(template_string)

In [37]:
# Para casos mas complejos: sistema, usuarios, asistente...etc
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un trabajador del campo, oriundo de la alpujarra de Granada."),
    ("human", "Traduce esto al {language}: {text}")
])

In [38]:
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un trabajador del campo, oriundo de la alpujarra de Granada.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Traduce esto al {language}: {text}'), additional_kwargs={})])

In [39]:
prompt_template.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un trabajador del campo, oriundo de la alpujarra de Granada.'), additional_kwargs={})

In [40]:
prompt_template.messages[1]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Traduce esto al {language}: {text}'), additional_kwargs={})

In [41]:
prompt_template.messages[1].prompt.input_variables

['language', 'text']

In [42]:
language = "Granaíno"
text = "¿De dónde vienes?"

translation_prompt = prompt_template.format_messages(
    language=language,
    text=text
    )

In [43]:
translation_prompt

[SystemMessage(content='Eres un trabajador del campo, oriundo de la alpujarra de Granada.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Traduce esto al Granaíno: ¿De dónde vienes?', additional_kwargs={}, response_metadata={})]

In [44]:
print(type(translation_prompt))
print(type(translation_prompt[0]))
print(type(translation_prompt[1]))

<class 'list'>
<class 'langchain_core.messages.system.SystemMessage'>
<class 'langchain_core.messages.human.HumanMessage'>


In [45]:
translator_response = llm.invoke(translation_prompt)

translator_response.content

'¿Ande vienes, chavá?'

In [52]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un noble de la realeza española del siglo XV."),
    ("human", "Traduce esto al {language}: {text}")
])

language = "Castellano antiguo"
text = "¿De dónde vienes?"

translation_prompt = prompt_template.format_messages(
    language=language,
    text=text
    )

customer_response = llm.invoke(translation_prompt)

customer_response.content

'¿De do vienes?'

## 2.3. - Output Parsers

En LangChain, los output parsers son herramientas clave para procesar y estructurar las respuestas que se obtienen de los modelos de lenguaje antes de ser utilizadas en otros pasos del flujo de trabajo. Estos parsers te permiten transformar la salida en bruto de los modelos en un formato más adecuado para tu aplicación, ya sea en forma de texto, datos estructurados o incluso en la ejecución de funciones específicas.

A continuación, una tabla con los output parsers más comunes en LangChain y su uso principal:

| **Output Parser**        | **¿Para qué sirve?**                                                                                                                                                 |
| ------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `JsonOutputParser`       | Convierte la salida del modelo en un objeto JSON (estructura de diccionario o lista). Ideal cuando esperas respuestas estructuradas como JSON o diccionarios.        |
| `RegexOutputParser`      | Extrae información utilizando expresiones regulares. Se usa cuando la respuesta del modelo sigue un patrón específico que se puede identificar con regex.            |
| `StructuredOutputParser` | Permite parsear la salida en estructuras de datos más complejas. Ideal para cuando necesitas que el modelo devuelva datos tabulares o jerárquicos.                   |
| `VariableParser`         | Analiza las respuestas del modelo en busca de valores de variables específicas. Utilizado cuando deseas extraer datos de las respuestas para usarlos en otros pasos. |
| `TextOutputParser`       | Convierte la salida en un texto plano procesable. Ideal cuando la salida del modelo es simplemente texto sin estructura.                                             |
| `SQLOutputParser`        | Convierte la salida del modelo en consultas SQL estructuradas. Es útil si el modelo está generando consultas a bases de datos.                                       |
| `PythonOutputParser`     | Convierte la salida del modelo en código Python ejecutable. Utilizado cuando necesitas generar código a partir de la respuesta del modelo.                           |
| `ActionOutputParser`     | Permite que la salida sea convertida en una acción específica o una ejecución de función. Ideal para flujos de trabajo más dinámicos donde se deben ejecutar tareas. |

In [53]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "muy asequible!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'muy asequible!'}

In [54]:
customer_review = """\
Este soplador de hojas es bastante increíble. Tiene cuatro configuraciones:\
soplador de vela, brisa suave, ciudad ventosa y tornado. \
Llegó en dos días, justo a tiempo para el regalo de aniversario de mi esposa. \
Creo que a mi esposa le gustó tanto que se quedó sin palabras. \
Hasta ahora he sido el único que lo ha usado, y lo he estado usando cada dos mañanas para limpiar las hojas de nuestro césped. \
Es ligeramente más caro que los otros sopladores de hojas que hay por ahí, pero creo que vale la pena por las características adicionales.
"""

review_template = """\
Para el siguiente texto, extrae la siguiente información:

gift: ¿Fue el artículo comprado como un regalo para otra persona? \
Responde True si es sí, False si no o si no se sabe.

delivery_days: ¿Cuántos días tardó en llegar el producto? \
Si esta información no se encuentra, devuelve -1.

price_value: Extrae cualquier frase sobre el valor o precio,\
y devuélvelas como una lista de Python separada por comas.

Formatea la salida como JSON con las siguientes claves:
gift
delivery_days
price_value

texto: {text}
"""

In [55]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Para el siguiente texto, extrae la siguiente información:\n\ngift: ¿Fue el artículo comprado como un regalo para otra persona? Responde True si es sí, False si no o si no se sabe.\n\ndelivery_days: ¿Cuántos días tardó en llegar el producto? Si esta información no se encuentra, devuelve -1.\n\nprice_value: Extrae cualquier frase sobre el valor o precio,y devuélvelas como una lista de Python separada por comas.\n\nFormatea la salida como JSON con las siguientes claves:\ngift\ndelivery_days\nprice_value\n\ntexto: {text}\n'), additional_kwargs={})]


In [56]:
prompt_template.messages[0].prompt.input_variables

['text']

In [57]:
messages = prompt_template.format_messages(text=customer_review)

response = llm.invoke(messages)

print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["ligeramente más caro"]
}


In [58]:
type(response.content)

str

In [59]:
# Si intentamos acceder a response como si fuera un diccionario...
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [60]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [69]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased\
    as a gift for someone else? \
    Answer True if yes,\
    False if not or unknown."
    )

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days\
    did it take for the product\
    to arrive? If this \
    information is not found,\
    output -1."
    )

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any\
    sentences about the value or \
    price, and output them as a \
    comma separated Python list."
    )

response_schemas = [
    gift_schema, 
    delivery_days_schema,
    price_value_schema
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [70]:
format_instructions = output_parser.get_format_instructions()

In [71]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased    as a gift for someone else?     Answer True if yes,    False if not or unknown.
	"delivery_days": string  // How many days    did it take for the product    to arrive? If this     information is not found,    output -1.
	"price_value": string  // Extract any    sentences about the value or     price, and output them as a     comma separated Python list.
}
```


In [72]:
review_template_2 = """\
Para el siguiente texto, extrae la siguiente información:

gift: ¿Fue el artículo comprado como un regalo para otra persona? \
Responde True si es sí, False si no o si no se sabe.

delivery_days: ¿Cuántos días tardó en llegar el producto? \
Si esta información no se encuentra, devuelve -1.

price_value: Extrae cualquier frase sobre el valor o el precio, \
y devuélvelas como una lista de Python separada por comas.

texto: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(
    text=customer_review, 
    format_instructions=format_instructions
    )


In [73]:
print(messages[0].content)

Para el siguiente texto, extrae la siguiente información:

gift: ¿Fue el artículo comprado como un regalo para otra persona? Responde True si es sí, False si no o si no se sabe.

delivery_days: ¿Cuántos días tardó en llegar el producto? Si esta información no se encuentra, devuelve -1.

price_value: Extrae cualquier frase sobre el valor o el precio, y devuélvelas como una lista de Python separada por comas.

texto: Este soplador de hojas es bastante increíble. Tiene cuatro configuraciones:soplador de vela, brisa suave, ciudad ventosa y tornado. Llegó en dos días, justo a tiempo para el regalo de aniversario de mi esposa. Creo que a mi esposa le gustó tanto que se quedó sin palabras. Hasta ahora he sido el único que lo ha usado, y lo he estado usando cada dos mañanas para limpiar las hojas de nuestro césped. Es ligeramente más caro que los otros sopladores de hojas que hay por ahí, pero creo que vale la pena por las características adicionales.


The output should be a markdown code sni

In [74]:
response = llm.invoke(messages)
print(response.content)

```json
{
	"gift": True,
	"delivery_days": 2,
	"price_value": ["Es ligeramente más caro que los otros sopladores de hojas que hay por ahí, pero creo que vale la pena por las características adicionales"]
}
```


In [79]:
output_dict = output_parser.parse(response.content)

ImportError: cannot import name 'OutputParserException' from 'langchain.output_parsers' (/home/gmachin/.local/share/virtualenvs/exploring-langchain-UOzDUSui/lib/python3.11/site-packages/langchain/output_parsers/__init__.py)

In [77]:
output_dict

NameError: name 'output_dict' is not defined

In [90]:
type(output_dict)

dict

In [91]:
output_dict.get('delivery_days')

2

LangChain:
- Model
- Prompts
- Output Parsers